In [2]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np


In [93]:
df_users = pd.read_csv('users.csv')
df_users = df_users.drop_duplicates(subset='username')
df_users.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4223 entries, 0 to 4467
Data columns (total 7 columns):
_id                       4223 non-null object
username                  4223 non-null object
uid                       4223 non-null object
attraction_review_name    4223 non-null object
nb_cities_visited         4223 non-null object
src                       4223 non-null int64
nb_contributions          4223 non-null object
dtypes: int64(1), object(6)
memory usage: 263.9+ KB


In [94]:
df_reviews = pd.read_excel('reviews_32618_for_1098_users_with_location.xlsx')
df_reviews.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32580 entries, 0 to 32579
Data columns (total 12 columns):
id              32580 non-null int64
username        32580 non-null object
type            32580 non-null object
date            32580 non-null datetime64[ns]
title           32580 non-null object
text            32579 non-null object
rating          32580 non-null int64
helpfulness     24105 non-null float64
total_points    32580 non-null int64
taObject        32580 non-null object
taObjectUrl     32580 non-null object
taObjectCity    32580 non-null object
dtypes: datetime64[ns](1), float64(1), int64(3), object(7)
memory usage: 3.0+ MB


In [95]:
df = df_reviews.pivot_table(index='username',columns=['taObjectCity'],values='rating',aggfunc=np.mean)

In [6]:
#df = df.fillna(0)
df.info()
df.iloc[[103],:]

<class 'pandas.core.frame.DataFrame'>
Index: 1098 entries, 422 to Farnborough
Columns: 5927 entries, Aachen to Г€ze
dtypes: float64(5927)
memory usage: 49.7+ MB


taObjectCity,Aachen,Aamby Valley City,Aarhus,Abbots Salford,Abbotsford,Aberaeron,Abercraf,Aberdeen,Aberdyfi (Aberdovey),Aberfeldy,...,Zipaquira,Zippori,Zoeterwoude,Zonnebeke,Zrece,Zurich,Zurs,Zwolle,Г–stersund,Г€ze
username,,,,,,,,,,,,,,,,,,,,,
ASWh,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0


In [7]:
count_list = []
for i in df.columns:
    count_list.append(df[i].sum())
print(np.mean(count_list))

12.648322906435409


In [8]:
df.head()

taObjectCity,Aachen,Aamby Valley City,Aarhus,Abbots Salford,Abbotsford,Aberaeron,Abercraf,Aberdeen,Aberdyfi (Aberdovey),Aberfeldy,...,Zipaquira,Zippori,Zoeterwoude,Zonnebeke,Zrece,Zurich,Zurs,Zwolle,Г–stersund,Г€ze
username,,,,,,,,,,,,,,,,,,,,,
422,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1771,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2093,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0
4038,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5264,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [96]:
df.loc[((df.loc[:,:]*df.loc["ASWh",:]).count(axis='columns')>1),:].info()

<class 'pandas.core.frame.DataFrame'>
Index: 488 entries, 422 to FanMan10
Columns: 5927 entries, Aachen to Г€ze
dtypes: float64(5927)
memory usage: 22.1+ MB


In [97]:
def find_n_nearest_neighbors(username,n=5):
    dict_neighbours = {}
    df_common = df.loc[((df.loc[:,:]*df.loc[username,:]).count(axis='columns')>1),:]
    df_common = df_common.loc[:,df_common.loc[username,df_common.loc[username,:]>0].index]
    for val,row in df_common.iterrows():
        set_rated = df_common.loc[[val,username],(df_common.loc[val,:]*df_common.loc[username,:])>0].columns
        sim = np.sum(df_common.loc[val,set_rated]*df_common.loc[username,set_rated])/np.sqrt((np.sum(df_common.loc[val,set_rated]**2))*(np.sum(df_common.loc[username,set_rated]**2)))
        dict_neighbours[val] = sim
    return dict_neighbours

In [98]:
dict_neighbours = find_n_nearest_neighbors('ASWh')

In [99]:
for key,item in dict_neighbours.items():
    print(str(key) + ' : ' + str(item))

422 : 0.9213907332595919
2093 : 0.9282581277609508
243345 : 0.9103664774626049
678855 : 0.9317550856793111
007solotraveler : 0.9993077047272493
10179k79 : 0.9761870601839526
124_10 : 0.999028234937562
1969ws : 0.9992467751496917
1975mark : 0.997785157856609
1984travel : 0.994623070946234
19katy80 : 0.9837174318379912
1Angie : 0.9938837346736188
1NicePerson : 0.994284170984449
1Vegas2008 : 0.9995120760870788
2008travellover : 0.9661807419087547
2011ChgoWanderer : 0.9999134461416996
23betty : 0.9843366432473037
25YearsAbroad : 0.999028234937562
29grapes : 0.9542095375510589
2Travelers2go : 0.9472174043614027
2toots : 0.9067473823021557
32369Terry : 0.9970544855015815
3kidsmama : 0.958663799040612
45Jen : 0.999028234937562
4959keeton : 0.9965960742937383
54Penny : 0.997785157856609
6079max : 0.9956241531340855
6dogs : 0.9337191657012796
7ContinentsX : 0.9978654315820765
A.Brouwer : 0.9817580328368792
AA75 : 0.9723299610314361
AANetherlands : 1.0
ABCDEFG_1234 : 0.904205749785131
ABitSpoile

In [100]:
df_pers = pd.read_excel('pers_scores_1098.xlsx',index_col = 'username')
df_pers = df_pers.subtract(df_pers.min(axis=0),axis=1).divide(df_pers.mean(axis=0)-df_pers.min(axis=0),axis=1)
df_pers_neighbours = df_pers.loc[dict_neighbours.keys(),:]
df_pers_neighbours = (df_pers_neighbours.loc['ASWh',:]*df_pers_neighbours).divide(np.sqrt((np.sum(df_pers_neighbours.loc['ASWh',:]**2))*((df_pers_neighbours**2).sum(axis=1))),axis=0).sum(axis=1)

In [101]:
df_sim = ((pd.Series(dict_neighbours) + df_pers_neighbours)/2).drop('ASWh')

In [102]:
print(df.loc[['AnaS1','ASWh'],(df.loc['AnaS1',:]*df.loc['ASWh',:])>0])

taObjectCity  New York City  Paris
username                          
AnaS1              4.500000    4.0
ASWh               4.666667    4.0


In [103]:
sorted_df_sim = df_sim.sort_values(ascending=False)[:10]
sorted_df_sim

AlisonandBill       0.999873
AnaS1               0.999818
Dodson              0.999806
AnniiL              0.999714
BradandJanet        0.999684
1Vegas2008          0.999633
F14Captain          0.999588
Dogfish60min        0.999532
2011ChgoWanderer    0.999524
Aidasas             0.999478
dtype: float64

In [107]:
neighbours = sorted_df_sim.index
sims = sorted_df_sim.values
condition = np.logical_or(df.loc[neighbours[1],:]>0,df.loc[neighbours[2],:]>0)
for i in range(3,len(neighbours)-1):
    condition = np.logical_or(condition,df.loc[neighbours[i],:]>0)
df_neighbours = df.loc[neighbours,condition]
df_prediction = df.loc['ASWh',:].mean() + df_neighbours.sub(df_neighbours.mean(axis='columns'),axis=0).multiply(sims,axis=0).sum().divide(df_neighbours.multiply(sims,axis=0).sum())
df_prediction.sort_values(ascending=False)

taObjectCity
Belgrave             4.144643
Killarney            4.144643
Ferntree Gully       4.144643
Gold Coast           4.118958
Barcelona            4.118958
Orange               4.118958
Nanaimo              4.080392
Stockholm            4.080392
Field                4.037667
Lake Louise          4.037667
Corolla              4.037667
Wheaton              4.037667
Tulsa                4.037667
Katoomba             4.035880
Montpellier          3.995238
New York City        3.987138
Rome                 3.987010
Santa Fe             3.974545
Century City         3.974545
Auckland             3.974545
Lahaina              3.974545
Amsterdam            3.974545
Weehawken            3.974545
Singapore            3.964137
Berlin               3.958095
Budapest             3.957948
Corvallis            3.956667
Tacoma               3.956667
Seattle              3.956667
Prague               3.956667
                       ...   
London               3.830529
East Glacier Park    3.8304

In [92]:
df_neighbours = df_neighbours.replace(0,np.nan)
df_prediction = df.loc['ASWh',:].replace(0,np.nan).mean() + df_neighbours.sub(df_neighbours.mean(axis='columns'),axis=0).multiply(sims,axis=0).sum().divide(df_neighbours.multiply(sims,axis=0).sum())
df_prediction.sort_values(ascending=False)

taObjectCity
Belgrave             4.144643
Killarney            4.144643
Ferntree Gully       4.144643
Gold Coast           4.118958
Barcelona            4.118958
Orange               4.118958
Nanaimo              4.080392
Stockholm            4.080392
Field                4.037667
Lake Louise          4.037667
Corolla              4.037667
Wheaton              4.037667
Tulsa                4.037667
Katoomba             4.035880
Montpellier          3.995238
New York City        3.987138
Rome                 3.987010
Santa Fe             3.974545
Century City         3.974545
Auckland             3.974545
Lahaina              3.974545
Amsterdam            3.974545
Weehawken            3.974545
Singapore            3.964137
Berlin               3.958095
Budapest             3.957948
Corvallis            3.956667
Tacoma               3.956667
Seattle              3.956667
Prague               3.956667
                       ...   
London               3.830529
East Glacier Park    3.8304